In [231]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier

In [232]:
#loading the dataset
df=pd.read_csv('/kaggle/input/human-resources-data-set/HRDataset_v14.csv')
df

,Employee_Name,EmpID,MarriedID,MaritalStatusID,GenderID,EmpStatusID,DeptID,PerfScoreID,FromDiversityJobFairID,Salary,Termd,PositionID,Position,State,Zip,DOB,Sex,MaritalDesc,CitizenDesc,HispanicLatino,RaceDesc,DateofHire,DateofTermination,TermReason,EmploymentStatus,Department,ManagerName,ManagerID,RecruitmentSource,PerformanceScore,EngagementSurvey,EmpSatisfaction,SpecialProjectsCount,LastPerformanceReview_Date,DaysLateLast30,Absences
0,"Adinolfi, Wilson K",10026,0,0,1,1,5,4,0,62506,0,19,Production Technician I,MA,1960,07/10/83,M,Single,US Citizen,No,White,7/5/2011,NaN,N/A-StillEmployed,Active,Production,Michael Albert,22.0,LinkedIn,Exceeds,4.60,5,0,1/17/2019,0,1
1,"Ait Sidi, Karthikeyan",10084,1,1,1,5,3,3,0,104437,1,27,Sr. DBA,MA,2148,05/05/75,M,Married,US Citizen,No,White,3/30/2015,6/16/2016,career change,Voluntarily Terminated,IT/IS,Simon Roup,4.0,Indeed,Fully Meets,4.96,3,6,2/24/2016,0,17
2,"Akinkuolie, Sarah",10196,1,1,0,5,5,3,0,64955,1,20,Production Technician II,MA,1810,09/19/88,F,Married,US Citizen,No,White,7/5/2011,9/24/2012,hours,Voluntarily Terminated,Production,Kissy Sullivan,20.0,LinkedIn,Fully Meets,3.02,3,0,5/15/2012,0,3
3,"Alagbe,Trina",10088,1,1,0,1,5,3,0,64991,0,19,Production Technician I,MA,1886,09/27/88,F,Married,US Citizen,No,White,1/7/2008,NaN,N/A-StillEmployed,Active,Production,Elijiah Gray,16.0,Indeed,Fully Meets,4.84,5,0,1/3/2019,0,15
4,"Anderson, Carol",10069,0,2,0,5,5,3,0,50825,1,19,Production Technician I,MA,2169,09/08/89,F,Divorced,US Citizen,No,White,7/11/2011,9/6/2016,return to school,Voluntarily Terminated,Production,Webster Butler,39.0,Google Search,Fully Meets,5.00,4,0,2/1/2016,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,"Woodson, Jason",10135,0,0,1,1,5,3,0,65893,0,20,Production Technician II,MA,1810,05/11/85,M,Single,US Citizen,No,White,7/7/2014,NaN,N/A-StillEmployed,Active,Production,Kissy Sullivan,20.0,LinkedIn,Fully Meets,4.07,4,0,2/28/2019,0,13
307,"Ybarra, Catherine",10301,0,0,0,5,5,1,0,48513,1,19,Production Technician I,MA,2458,05/04/82,F,Single,US Citizen,No,Asian,9/2/2008,9/29/2015,Another position,Voluntarily Terminated,Production,Brannon Miller,12.0,Google Search,PIP,3.20,2,0,9/2/2015,5,4
308,"Zamora, Jennifer",10010,0,0,0,1,3,4,0,220450,0,6,CIO,MA,2067,08/30/79,F,Single,US Citizen,No,White,4/10/2010,NaN,N/A-StillEmployed,Active,IT/IS,Janet King,2.0,Employee Referral,Exceeds,4.60,5,6,2/21/2019,0,16
309,"Zhou, Julia",10043,0,0,0,1,3,3,0,89292,0,9,Data Analyst,MA,2148,02/24/79,F,Single,US Citizen,No,White,3/30/2015,NaN,N/A-StillEmployed,Active,IT/IS,Simon Roup,4.0,Employee Referral,Fully Meets,5.00,3,5,2/1/2019,0,11


In [233]:
#getting preliminary information about the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 311 entries, 0 to 310
Data columns (total 36 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Employee_Name               311 non-null    object 
 1   EmpID                       311 non-null    int64  
 2   MarriedID                   311 non-null    int64  
 3   MaritalStatusID             311 non-null    int64  
 4   GenderID                    311 non-null    int64  
 5   EmpStatusID                 311 non-null    int64  
 6   DeptID                      311 non-null    int64  
 7   PerfScoreID                 311 non-null    int64  
 8   FromDiversityJobFairID      311 non-null    int64  
 9   Salary                      311 non-null    int64  
 10  Termd                       311 non-null    int64  
 11  PositionID                  311 non-null    int64  
 12  Position                    311 non-null    object 
 13  State                       311 non

In [234]:
#since sex and GenderID has same value encoded it is better to drop it
(df['Sex'].apply(lambda x:1 if x=="M " else 0)==df['GenderID']).all()

True

In [235]:
def encode_dates(df,columns_with_prefix):
    df=df.copy()
    
    for column,prefix in columns_with_prefix:
        df[column]=pd.to_datetime(df[column])
        df[prefix+'_year']=df[column].dt.year
        df[prefix+'_month']=df[column].dt.month
        df[prefix+'_day']=df[column].dt.day
        df=df.drop(column,axis=1)
    return df
    
def ordinal_encode(df,columns_with_ordering):
    df=df.copy()
    
    for column,ordering in columns_with_ordering:
        
        df[column]=df[column].apply(lambda x:ordering.index(x))
    return df
        
def onehot_encode(df,columns):
    df=df.copy()
    for column in columns:
        dummies=pd.get_dummies(df[column],prefix=column)
        df=pd.concat([df,dummies],axis=1)
        df=df.drop(column,axis=1)
    return df

In [257]:
def preprocess_inputs(df):
    df=df.copy()
    df=df.drop(['DateofTermination','TermReason'],axis=1)
    df=df.drop(['ManagerName','ManagerID','Employee_Name','EmpID'],axis=1)
    
    df=df.drop('Sex',axis=1)
    #date encoding
    
    date_columns=[('DOB','DOB'),('DateofHire','DOH'),('LastPerformanceReview_Date','LPR')]
    df=encode_dates(df,columns_with_prefix=date_columns)
    
    df=df.drop('EmploymentStatus',axis=1)
    
    #ordinal Encoding
    df=df.drop('PerfScoreID',axis=1)
    encoding_columns=[('PerformanceScore',['Needs Improvement','Fully Meets','Exceeds','PIP'])]
    df=ordinal_encode(df,encoding_columns)
    
    onehot_columns=[column for column in df.columns if df[column].dtype=='object']
    df=onehot_encode(df,onehot_columns)
    for column in df.columns:
        if df[column].dtype=='bool':
            df[column]=df[column].astype(int)
            
            
            
            
    y=df['Termd']
    x=df.drop('Termd',axis=1)
    
    
    x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.7,random_state=123,shuffle=True)
    
    scaler=StandardScaler()
    scaler.fit(x_train)
    
    x_train=pd.DataFrame(pd.DataFrame(x_train),columns=x_train.columns,index=x_train.index)
    x_test=pd.DataFrame(pd.DataFrame(x_test),columns=x_test.columns,index=x_test.index)
    
    
            
    return x_train,x_test,y_train,y_test

In [237]:
df['DateofTermination']

0            NaN
1      6/16/2016
2      9/24/2012
3            NaN
4       9/6/2016
         ...    
306          NaN
307    9/29/2015
308          NaN
309          NaN
310          NaN
Name: DateofTermination, Length: 311, dtype: object

In [238]:
df.columns

Index(['Employee_Name', 'EmpID', 'MarriedID', 'MaritalStatusID', 'GenderID',
       'EmpStatusID', 'DeptID', 'PerfScoreID', 'FromDiversityJobFairID',
       'Salary', 'Termd', 'PositionID', 'Position', 'State', 'Zip', 'DOB',
       'Sex', 'MaritalDesc', 'CitizenDesc', 'HispanicLatino', 'RaceDesc',
       'DateofHire', 'DateofTermination', 'TermReason', 'EmploymentStatus',
       'Department', 'ManagerName', 'ManagerID', 'RecruitmentSource',
       'PerformanceScore', 'EngagementSurvey', 'EmpSatisfaction',
       'SpecialProjectsCount', 'LastPerformanceReview_Date', 'DaysLateLast30',
       'Absences'],
      dtype='object')

In [239]:
df['PerformanceScore'].unique()

array(['Exceeds', 'Fully Meets', 'Needs Improvement', 'PIP'], dtype=object)

In [258]:
x_train,x_test,y_train,y_test=preprocess_inputs(df)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(217, 117)
(94, 117)
(217,)
(94,)


/tmp/ipykernel_30/1510455971.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[column]=pd.to_datetime(df[column])


# Training the Models

In [260]:
models={'K-Nearest Neighbors':KNeighborsClassifier(),
       'Logistic Regression':LogisticRegression(),
       'Support Vector Machine':SVC(),
       'Decision Tree':DecisionTreeClassifier(),
       'Neural Network':MLPClassifier()}

In [262]:
for name,model in models.items():
    print(name)
    model.fit(x_train,y_train)
    print(name+'_trained')
    print('The accuracy of the model:',model.score(x_test,y_test))
    

K-Nearest Neighbors
K-Nearest Neighbors_trained
The accuracy of the model: 0.48936170212765956
Logistic Regression
Logistic Regression_trained
The accuracy of the model: 0.8297872340425532
Support Vector Machine
Support Vector Machine_trained
The accuracy of the model: 0.6276595744680851
Decision Tree
Decision Tree_trained
The accuracy of the model: 0.9893617021276596
Neural Network
Neural Network_trained
The accuracy of the model: 0.46808510638297873


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [241]:
[column for column in x.columns if x[column].dtype=='object']

[]

In [242]:
#checking the length of unique value in dataset
{column:len(df[column].unique()) for column in df.columns}

{'Employee_Name': 311,
 'EmpID': 311,
 'MarriedID': 2,
 'MaritalStatusID': 5,
 'GenderID': 2,
 'EmpStatusID': 5,
 'DeptID': 6,
 'PerfScoreID': 4,
 'FromDiversityJobFairID': 2,
 'Salary': 308,
 'Termd': 2,
 'PositionID': 30,
 'Position': 32,
 'State': 28,
 'Zip': 158,
 'DOB': 307,
 'Sex': 2,
 'MaritalDesc': 5,
 'CitizenDesc': 3,
 'HispanicLatino': 4,
 'RaceDesc': 6,
 'DateofHire': 101,
 'DateofTermination': 97,
 'TermReason': 18,
 'EmploymentStatus': 3,
 'Department': 6,
 'ManagerName': 21,
 'ManagerID': 24,
 'RecruitmentSource': 9,
 'PerformanceScore': 4,
 'EngagementSurvey': 119,
 'EmpSatisfaction': 5,
 'SpecialProjectsCount': 9,
 'LastPerformanceReview_Date': 137,
 'DaysLateLast30': 7,
 'Absences': 20}

In [243]:
x.isna().sum()

MarriedID                                    0
MaritalStatusID                              0
GenderID                                     0
EmpStatusID                                  0
DeptID                                       0
                                            ..
RecruitmentSource_Indeed                     0
RecruitmentSource_LinkedIn                   0
RecruitmentSource_On-line Web application    0
RecruitmentSource_Other                      0
RecruitmentSource_Website                    0
Length: 118, dtype: int64

In [244]:
#dropping unneeded columns




In [245]:
#checking the length of unique value in dataset
{column:len(df[column].unique()) for column in df.columns}

{'Employee_Name': 311,
 'EmpID': 311,
 'MarriedID': 2,
 'MaritalStatusID': 5,
 'GenderID': 2,
 'EmpStatusID': 5,
 'DeptID': 6,
 'PerfScoreID': 4,
 'FromDiversityJobFairID': 2,
 'Salary': 308,
 'Termd': 2,
 'PositionID': 30,
 'Position': 32,
 'State': 28,
 'Zip': 158,
 'DOB': 307,
 'Sex': 2,
 'MaritalDesc': 5,
 'CitizenDesc': 3,
 'HispanicLatino': 4,
 'RaceDesc': 6,
 'DateofHire': 101,
 'DateofTermination': 97,
 'TermReason': 18,
 'EmploymentStatus': 3,
 'Department': 6,
 'ManagerName': 21,
 'ManagerID': 24,
 'RecruitmentSource': 9,
 'PerformanceScore': 4,
 'EngagementSurvey': 119,
 'EmpSatisfaction': 5,
 'SpecialProjectsCount': 9,
 'LastPerformanceReview_Date': 137,
 'DaysLateLast30': 7,
 'Absences': 20}

In [246]:
df['ManagerName'].unique()

array(['Michael Albert', 'Simon Roup', 'Kissy Sullivan', 'Elijiah Gray',
       'Webster Butler', 'Amy Dunn', 'Alex Sweetwater', 'Ketsia Liebig',
       'Brannon Miller', 'Peter Monroe', 'David Stanley', 'Kelley Spirea',
       'Brandon R. LeBlanc', 'Janet King', 'John Smith',
       'Jennifer Zamora', 'Lynn Daneault', 'Eric Dougall',
       'Debra Houlihan', 'Brian Champaigne', 'Board of Directors'],
      dtype=object)

In [247]:
df

,Employee_Name,EmpID,MarriedID,MaritalStatusID,GenderID,EmpStatusID,DeptID,PerfScoreID,FromDiversityJobFairID,Salary,Termd,PositionID,Position,State,Zip,DOB,Sex,MaritalDesc,CitizenDesc,HispanicLatino,RaceDesc,DateofHire,DateofTermination,TermReason,EmploymentStatus,Department,ManagerName,ManagerID,RecruitmentSource,PerformanceScore,EngagementSurvey,EmpSatisfaction,SpecialProjectsCount,LastPerformanceReview_Date,DaysLateLast30,Absences
0,"Adinolfi, Wilson K",10026,0,0,1,1,5,4,0,62506,0,19,Production Technician I,MA,1960,07/10/83,M,Single,US Citizen,No,White,7/5/2011,NaN,N/A-StillEmployed,Active,Production,Michael Albert,22.0,LinkedIn,Exceeds,4.60,5,0,1/17/2019,0,1
1,"Ait Sidi, Karthikeyan",10084,1,1,1,5,3,3,0,104437,1,27,Sr. DBA,MA,2148,05/05/75,M,Married,US Citizen,No,White,3/30/2015,6/16/2016,career change,Voluntarily Terminated,IT/IS,Simon Roup,4.0,Indeed,Fully Meets,4.96,3,6,2/24/2016,0,17
2,"Akinkuolie, Sarah",10196,1,1,0,5,5,3,0,64955,1,20,Production Technician II,MA,1810,09/19/88,F,Married,US Citizen,No,White,7/5/2011,9/24/2012,hours,Voluntarily Terminated,Production,Kissy Sullivan,20.0,LinkedIn,Fully Meets,3.02,3,0,5/15/2012,0,3
3,"Alagbe,Trina",10088,1,1,0,1,5,3,0,64991,0,19,Production Technician I,MA,1886,09/27/88,F,Married,US Citizen,No,White,1/7/2008,NaN,N/A-StillEmployed,Active,Production,Elijiah Gray,16.0,Indeed,Fully Meets,4.84,5,0,1/3/2019,0,15
4,"Anderson, Carol",10069,0,2,0,5,5,3,0,50825,1,19,Production Technician I,MA,2169,09/08/89,F,Divorced,US Citizen,No,White,7/11/2011,9/6/2016,return to school,Voluntarily Terminated,Production,Webster Butler,39.0,Google Search,Fully Meets,5.00,4,0,2/1/2016,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,"Woodson, Jason",10135,0,0,1,1,5,3,0,65893,0,20,Production Technician II,MA,1810,05/11/85,M,Single,US Citizen,No,White,7/7/2014,NaN,N/A-StillEmployed,Active,Production,Kissy Sullivan,20.0,LinkedIn,Fully Meets,4.07,4,0,2/28/2019,0,13
307,"Ybarra, Catherine",10301,0,0,0,5,5,1,0,48513,1,19,Production Technician I,MA,2458,05/04/82,F,Single,US Citizen,No,Asian,9/2/2008,9/29/2015,Another position,Voluntarily Terminated,Production,Brannon Miller,12.0,Google Search,PIP,3.20,2,0,9/2/2015,5,4
308,"Zamora, Jennifer",10010,0,0,0,1,3,4,0,220450,0,6,CIO,MA,2067,08/30/79,F,Single,US Citizen,No,White,4/10/2010,NaN,N/A-StillEmployed,Active,IT/IS,Janet King,2.0,Employee Referral,Exceeds,4.60,5,6,2/21/2019,0,16
309,"Zhou, Julia",10043,0,0,0,1,3,3,0,89292,0,9,Data Analyst,MA,2148,02/24/79,F,Single,US Citizen,No,White,3/30/2015,NaN,N/A-StillEmployed,Active,IT/IS,Simon Roup,4.0,Employee Referral,Fully Meets,5.00,3,5,2/1/2019,0,11


In [248]:
df.isna().sum()

Employee_Name                   0
EmpID                           0
MarriedID                       0
MaritalStatusID                 0
GenderID                        0
EmpStatusID                     0
DeptID                          0
PerfScoreID                     0
FromDiversityJobFairID          0
Salary                          0
Termd                           0
PositionID                      0
Position                        0
State                           0
Zip                             0
DOB                             0
Sex                             0
MaritalDesc                     0
CitizenDesc                     0
HispanicLatino                  0
RaceDesc                        0
DateofHire                      0
DateofTermination             207
TermReason                      0
EmploymentStatus                0
Department                      0
ManagerName                     0
ManagerID                       8
RecruitmentSource               0
PerformanceSco